# Redis Stream Examples

## basic config

In [1]:
redis_host = "redis"
stream_key = "skey"
stream2_key = "s2key"

## connection

In [2]:
import redis
from time import time

r = redis.Redis( redis_host )
r.ping()

True

## xadd and xread

### add some data to the stream

In [3]:
for i in range(0,10):
    r.xadd( stream_key, { 'ts': time(), 'v': i } )
print( f"stream length: {r.xlen( stream_key )}")

stream length: 10


### read some data from the stream

In [4]:
## read 2 entries from stream_key
l = r.xread( count=2, streams={stream_key:0} )
print(l)

[[b'skey', [(b'1655660943841-0', {b'ts': b'1655660943.8354728', b'v': b'0'}), (b'1655660943843-0', {b'ts': b'1655660943.8435743', b'v': b'1'})]]]


### extract data from the returned structure

In [5]:
first_stream = l[0]
print( f"got data from stream: {first_stream[0]}")
fs_data = first_stream[1]
for id, value in fs_data:
    print( f"id: {id} value: {value[b'v']}")

got data from stream: b'skey'
id: b'1655660943841-0' value: b'0'
id: b'1655660943843-0' value: b'1'


### read more data from the stream
if we call the `xread` with the same arguments we will get the same data

In [6]:
l = r.xread( count=2, streams={stream_key:0} )
for id, value in l[0][1]:
    print( f"id: {id} value: {value[b'v']}")

id: b'1655660943841-0' value: b'0'
id: b'1655660943843-0' value: b'1'


to get new data we need to change the key passed to the call

In [7]:
last_id_returned = l[0][1][-1][0]
l = r.xread( count=2, streams={stream_key: last_id_returned} )
for id, value in l[0][1]:
    print( f"id: {id} value: {value[b'v']}")

id: b'1655660943844-0' value: b'2'
id: b'1655660943844-1' value: b'3'


In [8]:
last_id_returned = l[0][1][-1][0]
l = r.xread( count=2, streams={stream_key: last_id_returned} )
for id, value in l[0][1]:
    print( f"id: {id} value: {value[b'v']}")

id: b'1655660943845-0' value: b'4'
id: b'1655660943845-1' value: b'5'


to get only newer entries

In [9]:
print( f"stream length: {r.xlen( stream_key )}")
# wait for 5s for new messages
l = r.xread( count=1, block=5000, streams={stream_key: '$'} )
print( f"after 5s block, got an empty list {l}, no *new* messages on the stream")
print( f"stream length: {r.xlen( stream_key )}")

stream length: 10
after 5s block, got an empty list [], no *new* messages on the stream
stream length: 10


### 2nd stream
Add some messages to a 2nd stream

In [10]:
for i in range(1000,1010):
    r.xadd( stream2_key, { 'v': i } )
print( f"stream length: {r.xlen( stream2_key )}")

stream length: 10


get messages from the 2 streams

In [11]:
l = r.xread( count=1, streams={stream_key:0,stream2_key:0} )
for k,d in l:
    print(f"got from {k} the entry {d}")

got from b'skey' the entry [(b'1655660943841-0', {b'ts': b'1655660943.8354728', b'v': b'0'})]
got from b's2key' the entry [(b'1655660973919-0', {b'v': b'1000'})]
